<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/Summorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bert-extractive-summarizer
!pip install -U transformers

     |████████████████████████████████| 1.9MB 32.9MB/s 
     |████████████████████████████████| 3.2MB 45.4MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7b938be79e5a9f89c6650998c702c0a96c4ec82531362e41f31129635e053a0d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.3.3)


In [20]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-03-14 05:07:09--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm910.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-03-14 05:07:09--  http://cache-mskm910.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm910.cdn.yandex.net (cache-mskm910.cdn.yandex.net)... 5.45.220.20, 2a02:6b8:0:2002::1b
Connecting to cache-mskm910.cdn.yandex.net (cache-mskm910.cdn.yandex.net)|5.45.220.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [3]:
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEH7KE4SCWPI626GW46TAKTP36
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH7ZDAUWRJUVRBMMYLDAKTP7Q
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkrText.csv?token=AITWEH6BLDWIZWOY6OWA6T3AKTP7W

--2021-03-14 04:52:29--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEH7KE4SCWPI626GW46TAKTP36
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30447 (30K) [text/plain]
Saving to: ‘people.csv?token=AITWEH7KE4SCWPI626GW46TAKTP36’

people.csv?token=AI 100%[===================>]  29.73K  --.-KB/s    in 0s      

2021-03-14 04:52:29 (125 MB/s) - ‘people.csv?token=AITWEH7KE4SCWPI626GW46TAKTP36’ saved [30447/30447]

--2021-03-14 04:52:29--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH7ZDAUWRJUVRBMMYLDAKTP7Q
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserconten

In [21]:
import requests
import pandas as pd
from summarizer import Summarizer
from transformers import BertTokenizer, BertModel, BertConfig
from tqdm import tqdm

import torch

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import string

from pymystem3 import Mystem
import re
import math

nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
russian_stopwords.append('рис')
russian_stopwords.append('т.к.')
russian_stopwords.append('пермь')
russian_stopwords.append('аннотация')
russian_stopwords.append('оглавление')
russian_stopwords.append('год')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
vkr = pd.read_csv('/content/vkr.csv?token=AITWEH7ZDAUWRJUVRBMMYLDAKTP7Q')
vkrText = pd.read_csv('/content/vkrText.csv?token=AITWEH6BLDWIZWOY6OWA6T3AKTP7W')
vkrText.head()

,Unnamed: 0,Unnamed: 0.1,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file,fullText,shortText
0,0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,NaN,NaN,NaN
1,1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...,\n ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕНН...,NaN
2,2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
3,3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
4,4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
custom_model.eval()
custom_model.to('cuda')

In [22]:
def lowerCase(Corpus):
    corpus = "".join([i.lower().replace('\n',' ') for i in Corpus])
    return corpus

PUNCT_TO_REMOVE = "'#$%&\()*+-/<=>|_[]{}*/\",:"
def remove_punctuation(Corpus):
    """custom function to remove the punctuation"""
    return "".join([i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) for i in Corpus])

def remove_stopwords(Corpus):
    """custom function to remove the stopwords"""
    return " ".join([token for token in Corpus.split() if token not in russian_stopwords])

def remove_freqwords(Corpus):
    cnt = Counter()
    for text in Corpus:
        for word in text.split():
            cnt[word] += 1
    FREQWORDS = set([w for (w, wc) in cnt.most_common(30)])
    return " ".join([word for word in str(Corpus).split() if word not in FREQWORDS])

def remove_urls(Corpus):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    return_text=[]
    for text in Corpus:
        text=url_pattern.sub(r'', text)
        text=html_pattern.sub(r'',text)
        return_text.append(text)
    return "".join(i for i in return_text)
    
def remove_numbers(Corpus):
    text = ["".join(i for i in text if not i.isdigit()) for text in Corpus]
    return "".join(i for i in text)

def Stem(Corpus):
    stemmer = SnowballStemmer("russian") 
    return "".join(m.lemmatize(Corpus))

def Lemm(Corpus):
    m = Mystem()
    return "".join(m.lemmatize(Corpus))

def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 

def clearTextAfterGarbage(Corpus):
    corpus = Corpus
    corpus = lowerCase(Corpus)
    corpus = remove_punctuation(corpus)
    corpus = remove_urls(corpus)
    corpus = remove_numbers(corpus)
    corpus = remove_stopwords(corpus)
    corpus = Lemm(corpus)
    #corpus = Stem(corpus)
    corpus = remove_freqwords(corpus)
    return corpus

In [23]:
text = clearTextAfterGarbage(vkrText.loc[290, 'fullText'])
text

'работа описывать процесс реализация виртуальный собеседник помощь лексикосинтаксический шаблон использование язык lspl. разрабатывать виртуальный собеседник реализовать система администрирование. работа содержать глава иллюстрация приложение. введение современный продуктивный обучение предполагать интенсивный работа сторона студент сторона преподаватель число выполнение большой количество проект усвоение получать знание практика. зачастую такой проект требовать большой затрата время сторона преподаватель сторона студент. случай такой проект должный автоматизировать хотя частично. автоматизация один такой учебный проект пойти речь работа. ход работа улучшать разрабатывать ранее виртуальный собеседник деловой игра «развертывание» технология msf. ход деловой игра студент делиться группа группа должный опрашивать сотрудник отдел фирма филиал четыре город разбрасывать весь мир. каждый сотрудник обладать свой компетенция знать весь информация филиал сотрудник один филиал сумма обладать инфо

In [ ]:
vkrText.loc[290, :]

In [24]:
vkrText.head()

,Unnamed: 0,Unnamed: 0.1,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file,fullText,shortText
0,0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,NaN,NaN,NaN
1,1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...,\n ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕНН...,NaN
2,2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
3,3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
4,4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN


In [25]:
%%time
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = modelSum(text)
summary = "".join(result)
print(len(summary), summary)
#result = modelSum.run_embeddings(text, aggregate='mean')
#result.shape

8109 работа описывать процесс реализация виртуальный собеседник помощь лексикосинтаксический шаблон использование язык lspl. зачастую такой проект требовать большой затрата время сторона преподаватель сторона студент. ход деловой игра студент делиться группа группа должный опрашивать сотрудник отдел фирма филиал четыре город разбрасывать весь мир. тестирование показывать качество работа собеседник недостаточно высоко. научный новизна работа улучшение ранее разрабатывать метод решение проблема именно внедрение учебный процесс виртуальный собеседник. глава описание предметный область первый глава описывать деловой игра технология msf «развертывание» выявлять недостаток предыдущий версия быть сформировывать требование новый версия система. этап разработка писаться код приложение подготавливаться документация. этап проводиться тестирование приложение устраняться обнаруживать ошибка. деловой игра «развертывание» изучение технология msf ниу вшэпермь использоваться ряд деловой игра envisionpl

In [ ]:
with open("Output.txt", "w") as text_file:    
    text_file.write(summary)

## independent works

In [ ]:
%%time
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
for row in tqdm(range(vkrText.shape[0])):
    if type(vkrText.loc[row, 'fullText'])==str:
        #print(row, vkrText.loc[row, 'fullText'])
        text = clearTextAfterGarbage(vkrText.loc[row, 'fullText'])
        result = modelSum(text, min_length=20, max_length=100)
        summary = "".join(result)
        vkrText.loc[row, 'shortText'] = summary

In [ ]:
vkrText['shortText'].nunique()

In [ ]:
vkrText.head(10)

In [ ]:
vkrText.drop(['Unnamed: 0','Unnamed: 0.1','fullText'], axis=1, inplace=True)

In [ ]:
vkrText.to_csv('vkrTextShort.csv')

## Works grouped by Prof

In [26]:
clearVkr = vkrText.loc[vkrText["fullText"].dropna().index, :]
texts = clearVkr.groupby("name_prof")['fullText'].sum()

In [27]:
df = pd.DataFrame(texts)
df['shortText']=''
df['embeddings']=''
df.head()

,fullText,shortText,embeddings
name_prof,,,
Абашева Александра Сергеевна,\n ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕНН...,,
Андреева Ольга Юрьевна,ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО...,,
Артемьев Дмитрий Геннадьевич,\n\nПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕ...,,
Барахас Анхель,FEDERAL STATE EDUCATIONAL INSTITUTION OF HIGHE...,,
Белых Светлана Александровна,ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО...,,


In [28]:
%%time
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
i = 0
for row in tqdm(df.index):
#for row in df.index:
    #print(row)
    text = clearTextAfterGarbage(df.loc[row, 'fullText'])[:1000000]
    result = modelSum(text, min_length=20, max_length=100)
    summary = "".join(result)
    df.loc[row, 'shortText'] = summary
    embeddings = modelSum.run_embeddings(text, aggregate='mean')
    df.loc[row, 'embeddings'] = embeddings
    i+=1

100%|██████████| 60/60 [6:06:06<00:00, 366.10s/it]

CPU times: user 6h 29min 22s, sys: 3h 24min 38s, total: 9h 54min 1s
Wall time: 6h 6min 6s


In [29]:
df.head(10)

,fullText,shortText,embeddings
name_prof,,,
Абашева Александра Сергеевна,\n ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕНН...,абашев цель работа являться оценка эффективнос...,"[-0.4780663, -0.6637962, 0.6672945, -0.1769247..."
Андреева Ольга Юрьевна,ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО...,сбор данный проводить опрос потребитель менедж...,"[-0.49407846, -0.59260714, 0.64088005, -0.1710..."
Артемьев Дмитрий Геннадьевич,\n\nПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕ...,цель исследование являться снижение общий затр...,"[-0.54530317, -0.5717621, 0.67262906, -0.15913..."
Барахас Анхель,FEDERAL STATE EDUCATIONAL INSTITUTION OF HIGHE...,however at the moment there is still no resear...,"[-1.1577483, -0.3386782, 0.8410521, 0.0707182,..."
Белых Светлана Александровна,ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО...,приложение ……………………………………………………………………. задача ...,"[-0.49121606, -0.6550153, 0.682922, -0.1549412..."
Божья-Воля Анастасия Александровна,Пермский филиал федерального государственного ...,юрий михайлович плюснин научный руководитель к...,"[-0.57298595, -0.55314493, 0.6789771, -0.13729..."
Бузмаков Алексей Владимирович,"Пермь, 2018 год \n Аннотация В этой работе опи...",разрабатывать виртуальный собеседник реализова...,"[-0.3354775, -0.5867729, 0.6211972, -0.1910156..."
Букина Татьяна Витальевна,ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО ...,изучаться регионва период год год. исследовани...,"[-0.57718694, -0.6300543, 0.58949316, -0.13322..."
Быкова Анна Андреевна,ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО ...,работа использовать выборка российский банк пе...,"[-0.5914128, -0.56986386, 0.5640466, -0.110308..."


In [32]:
df.drop("fullText",axis=1, inplace=True)

In [33]:
df.to_csv("text1.csv")